In [1]:
import numpy as np
import re
import multiprocessing
import seaborn as sns
import pandas as pd
from matplotlib import pyplot as plt
from time import time
from gensim.models import Word2Vec
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import seaborn as sns
from tensorflow.keras.preprocessing.text import text_to_word_sequence, Tokenizer
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences,TimeseriesGenerator
import nltk 
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')
nltk.download('stopwords')
from nltk.stem import LancasterStemmer, WordNetLemmatizer
sns.set_style("darkgrid")
%matplotlib inline


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Sveta\AppData\Roaming\nltk_data...
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Sveta\AppData\Roaming\nltk_data...
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Sveta\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Sveta\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [2]:
data=pd.read_csv('data.csv')
data=data.drop(['Unnamed: 0'],axis=1)

for i in range(data.shape[0]):
    text=str(data['text'][i])

def remove_punctuation(s):
    for p in '!"#$%&\'()*+,-–./:;<=>?@[\\]^_`{|}~':
        s = s.replace(p, '')
    if len(s) == 0:
        return ' '
    return s
data['text'] = data['text'].apply(remove_punctuation)

In [3]:
dataset=np.array(data)
text = np.array(data['text'])
print(text[228])

Сегодня ваш мозг выдает гениальные идеи со скоростью света но не все они кстати и своевременны К сожалению для многих из них пока нет нужных ресурсов и это может не только печалить но и энергетически опустошать Постарайтесь записывать их чтобы в будущем когда появятся возможности реализовать Полезно меньше общаться  сейчас вас может разочаровать даже любимый человек


In [4]:
def processText(data):
    tokens=[]
    for line in data:
        newToken=text_to_word_sequence(line,filters='!"#$%&amp;()*+,-./:;&lt;=>?@[\\]^_`{|}~\t\n\ufeff',
                                  lower=True,split=' ')
        tokens.append(newToken)
    return tokens

data['tokens'] = processText(text)
data

,date,sign,text,tokens
0,2021-01-01,aries,Вы уверенно ступили в новый 2021 год и стоит п...,"[вы, уверенно, ступили, в, новый, 2021, год, и..."
1,2021-01-01,gemini,Продолжайте отмечать праздник так как вам комф...,"[продолжайте, отмечать, праздник, так, как, ва..."
2,2021-01-01,taurus,Хватит думать о работе планах на новый год и с...,"[хватит, думать, о, работе, планах, на, новый,..."
3,2021-01-01,cancer,Наконец вы можете выдохнуть Новый год прошел н...,"[наконец, вы, можете, выдохнуть, новый, год, п..."
4,2021-01-01,leo,Настраивайтесь на приятные впечатления сегодн...,"[настраивайтесь, на, приятные, впечатления, се..."
...,...,...,...,...
2526,2022-02-13,scorpio,Вы явно встанете не с той ноги и настроение и...,"[вы, явно, встанете, не, с, той, ноги, и, наст..."
2527,2022-02-13,sagittarius,Вам не хватает ярких эмоций и перемен Не стоит...,"[вам, не, хватает, ярких, эмоций, и, перемен, ..."
2528,2022-02-13,capricorn,Судьба сегодня проверит ваши нервы на прочност...,"[судьба, сегодня, проверит, ваши, нервы, на, п..."
2529,2022-02-13,aquarius,Контроль сегодня не помешает Держите в поле зр...,"[контроль, сегодня, не, помешает, держите, в, ..."


In [5]:
def stopword(tokens):
  # clean = []
  for line in tokens:
    clean = []
    for i in line:
      if i not in stopwords.words('russian'):
        clean.append(i)
    yield clean
    
clear_tokens = list(stopword(data['tokens']))
data['clear_tokens'] = clear_tokens

In [6]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [7]:
def lemmatize(text):
    for line in clear_tokens:
        res = list()
        for word in line:
            p = morph.parse(word)[0]
            res.append(p.normal_form)
        yield res

data['lemm_tokens'] = list(lemmatize(data['clear_tokens']))
data

,date,sign,text,tokens,clear_tokens,lemm_tokens
0,2021-01-01,aries,Вы уверенно ступили в новый 2021 год и стоит п...,"[вы, уверенно, ступили, в, новый, 2021, год, и...","[уверенно, ступили, новый, 2021, год, стоит, п...","[уверенно, ступить, новый, 2021, год, стоить, ..."
1,2021-01-01,gemini,Продолжайте отмечать праздник так как вам комф...,"[продолжайте, отмечать, праздник, так, как, ва...","[продолжайте, отмечать, праздник, комфортно, с...","[продолжать, отмечать, праздник, комфортно, ст..."
2,2021-01-01,taurus,Хватит думать о работе планах на новый год и с...,"[хватит, думать, о, работе, планах, на, новый,...","[хватит, думать, работе, планах, новый, год, с...","[хватить, думать, работа, план, новый, год, се..."
3,2021-01-01,cancer,Наконец вы можете выдохнуть Новый год прошел н...,"[наконец, вы, можете, выдохнуть, новый, год, п...","[можете, выдохнуть, новый, год, прошел, ура, г...","[мочь, выдохнуть, новый, год, пройти, ура, гос..."
4,2021-01-01,leo,Настраивайтесь на приятные впечатления сегодн...,"[настраивайтесь, на, приятные, впечатления, се...","[настраивайтесь, приятные, впечатления, сегодн...","[настраиваться, приятный, впечатление, сегодня..."
...,...,...,...,...,...,...
2526,2022-02-13,scorpio,Вы явно встанете не с той ноги и настроение и...,"[вы, явно, встанете, не, с, той, ноги, и, наст...","[явно, встанете, той, ноги, настроение, силы, ...","[явно, встать, тот, нога, настроение, сила, бы..."
2527,2022-02-13,sagittarius,Вам не хватает ярких эмоций и перемен Не стоит...,"[вам, не, хватает, ярких, эмоций, и, перемен, ...","[хватает, ярких, эмоций, перемен, стоит, ждать...","[хватать, яркий, эмоция, перемена, стоить, жда..."
2528,2022-02-13,capricorn,Судьба сегодня проверит ваши нервы на прочност...,"[судьба, сегодня, проверит, ваши, нервы, на, п...","[судьба, сегодня, проверит, ваши, нервы, прочн...","[судьба, сегодня, проверить, ваш, нерв, прочно..."
2529,2022-02-13,aquarius,Контроль сегодня не помешает Держите в поле зр...,"[контроль, сегодня, не, помешает, держите, в, ...","[контроль, сегодня, помешает, держите, поле, з...","[контроль, сегодня, помешать, держать, поле, з..."


In [8]:
crop_data = data.drop(['text', 'tokens', 'clear_tokens'], axis = 1)
crop_data.head()

,date,sign,lemm_tokens
0,2021-01-01,aries,"[уверенно, ступить, новый, 2021, год, стоить, ..."
1,2021-01-01,gemini,"[продолжать, отмечать, праздник, комфортно, ст..."
2,2021-01-01,taurus,"[хватить, думать, работа, план, новый, год, се..."
3,2021-01-01,cancer,"[мочь, выдохнуть, новый, год, пройти, ура, гос..."
4,2021-01-01,leo,"[настраиваться, приятный, впечатление, сегодня..."


In [9]:
crop_data.to_csv('clean_data.csv') 


###  Tf - idf

In [10]:
crop_data=pd.read_csv('clean_data.csv')

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(crop_data['lemm_tokens'])
tfidf_tokens = vectorizer.get_feature_names()
df_tfidfvect = pd.DataFrame(data = X.toarray(),index = range(len(crop_data)),columns = tfidf_tokens)

c:\Users\Sveta\projects\astro-main\text_venv\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
